# UDA-Hub: Multi-Agent Customer Support System

**Architecture**: Hierarchical (Supervisor) with 4 agents

This notebook demonstrates:
1. End-to-end workflow with structured logging
2. Tool usage (db_tools) during ticket processing
3. Confidence-based escalation from real similarity scores
4. Session memory inspection by thread_id
5. Long-term memory persistence
6. Edge-case handling (low/no KB match → escalation)

## 1. Setup

In [ ]:
from dotenv import load_dotenv
load_dotenv()

import logging
logging.getLogger().setLevel(logging.INFO)

print("✅ Environment loaded")

In [ ]:
from agentic.workflow import orchestrator, get_session_state, CONFIDENCE_THRESHOLD
from langchain_core.messages import HumanMessage

print("✅ Orchestrator loaded")
print(f"✅ Confidence threshold: {CONFIDENCE_THRESHOLD:.0%}")
print(f"\nWorkflow nodes: {list(orchestrator.get_graph().nodes.keys())}")

In [ ]:
def ask(message: str, thread_id: str = "demo", ticket_id: str = "") -> dict:
    """Send a message and get structured response."""
    result = orchestrator.invoke(
        input={
            "messages": [HumanMessage(content=message)],
            "ticket_id": ticket_id or thread_id,
            "tools_invoked": [],
        },
        config={"configurable": {"thread_id": thread_id}}
    )
    return {
        "response": result["messages"][-1].content,
        "category": result.get("category", "unknown"),
        "escalated": result.get("needs_escalation", False),
        "confidence": result.get("knowledge_confidence", 0.0),
        "tools_used": result.get("tools_invoked", [])
    }

print("✅ Helper ready")

---
## 2. Tool Usage Demo (db_tools)

This demonstrates `get_user_info` and `get_user_reservations` being invoked during workflow.

**Watch the structured logs** showing: tool_invoked → tool_result

In [ ]:
# First, let's see what users exist in the database
from agentic.tools.db_tools import get_user_info, get_user_reservations, check_subscription_status

print("=" * 60)
print("DIRECT TOOL INVOCATION TEST")
print("=" * 60)

# Test get_user_info tool
print("\n📧 Testing get_user_info('alice@example.com'):")
print("-" * 40)
result = get_user_info.invoke("alice@example.com")
print(result)

In [ ]:
# Test get_user_reservations tool
print("\n📅 Testing get_user_reservations('alice@example.com'):")
print("-" * 40)
result = get_user_reservations.invoke("alice@example.com")
print(result)

In [ ]:
# Test check_subscription_status tool
print("\n💳 Testing check_subscription_status('alice@example.com'):")
print("-" * 40)
result = check_subscription_status.invoke("alice@example.com")
print(result)

In [ ]:
# Now test tools being invoked DURING workflow
# The email in the message triggers tool calls
print("\n" + "=" * 60)
print("TOOL USAGE DURING WORKFLOW")
print("=" * 60)
print("\n🔍 Watch the JSON logs below showing tool invocations:\n")

result = ask(
    "Hi, I'm alice@example.com and I need help with my subscription",
    thread_id="tool-demo-1"
)

print("\n" + "=" * 60)
print("RESULT SUMMARY")
print("=" * 60)
print(f"Category: {result['category']}")
print(f"Confidence: {result['confidence']:.0%}")
print(f"Escalated: {result['escalated']}")
print(f"Tools Used: {result['tools_used']}")
print(f"\nResponse: {result['response'][:300]}...")

---
## 3. Edge-Case Demo: Low Confidence → Escalation

This demonstrates that queries with low KB match confidence automatically escalate.

**Key**: We use real Chroma similarity scores (not string heuristics).

In [ ]:
# First, test a COMMON query that should have HIGH confidence
print("=" * 60)
print("TEST 1: Common Query (High Confidence Expected)")
print("=" * 60)
print("Query: 'How do I reset my password?'\n")

result_common = ask("How do I reset my password?", thread_id="edge-case-1")

print(f"\n✅ Confidence: {result_common['confidence']:.0%}")
print(f"✅ Threshold: {CONFIDENCE_THRESHOLD:.0%}")
print(f"✅ Escalated: {result_common['escalated']}")
print(f"✅ Category: {result_common['category']}")

In [ ]:
# Now test an OBSCURE query that should have LOW confidence and trigger escalation
print("\n" + "=" * 60)
print("TEST 2: Obscure Query (Low Confidence → Escalation)")
print("=" * 60)
print("Query: 'I need help with the enterprise API webhook integration for B2B partners'\n")

result_obscure = ask(
    "I need help with the enterprise API webhook integration for B2B partners",
    thread_id="edge-case-2"
)

print(f"\n⚠️ Confidence: {result_obscure['confidence']:.0%}")
print(f"⚠️ Threshold: {CONFIDENCE_THRESHOLD:.0%}")
print(f"⚠️ Escalated: {result_obscure['escalated']}")
print(f"⚠️ Category: {result_obscure['category']}")

if result_obscure['escalated']:
    print("\n✅ SUCCESS: Low confidence query correctly triggered escalation!")
else:
    print("\n❌ Query did not escalate - confidence may be above threshold")

In [ ]:
# Compare the two results
print("\n" + "=" * 60)
print("COMPARISON: Common vs Obscure Query")
print("=" * 60)
print(f"\n{'Query Type':<20} {'Confidence':<15} {'Escalated':<12} {'Category'}")
print("-" * 60)
print(f"{'Common (password)':<20} {result_common['confidence']:.0%}{'':>10} {str(result_common['escalated']):<12} {result_common['category']}")
print(f"{'Obscure (API)':<20} {result_obscure['confidence']:.0%}{'':>10} {str(result_obscure['escalated']):<12} {result_obscure['category']}")
print(f"\nThreshold: {CONFIDENCE_THRESHOLD:.0%}")

---
## 4. Session Memory Inspection

Demonstrates inspecting stored session state by `thread_id`.

In [ ]:
import uuid

# Create a new session with multiple messages
session_id = f"session-inspect-{uuid.uuid4().hex[:8]}"
print(f"Session ID: {session_id}")
print("=" * 60)

# First message
print("\n[Message 1] User: I'm having trouble with my subscription")
result1 = ask("I'm having trouble with my subscription", thread_id=session_id)
print(f"Agent: {result1['response'][:150]}...")

In [ ]:
# Second message in same session
print(f"\n[Message 2] User: What benefits do I get?")
result2 = ask("What benefits do I get?", thread_id=session_id)
print(f"Agent: {result2['response'][:150]}...")

In [ ]:
# Now inspect the session state
print("\n" + "=" * 60)
print("SESSION STATE INSPECTION")
print("=" * 60)
print(f"\nRetrieving state for thread_id: {session_id}\n")

state = get_session_state(session_id)

if state:
    print("📋 Stored Session State:")
    print("-" * 40)
    print(f"Category: {state['category']}")
    print(f"Needs Escalation: {state['needs_escalation']}")
    print(f"Confidence: {state['knowledge_confidence']:.0%}" if state['knowledge_confidence'] else "N/A")
    print(f"Tools Invoked: {state['tools_invoked']}")
    print(f"\n📨 Messages in Session ({len(state['messages'])}):") 
    for i, msg in enumerate(state['messages'], 1):
        role = msg['role'].upper()
        content = msg['content'][:80] + "..." if len(msg['content']) > 80 else msg['content']
        print(f"  {i}. [{role}]: {content}")
else:
    print("❌ No state found for this session")

---
## 5. Long-Term Memory Demo

Demonstrates cross-session memory persistence using `ticket_id`.

In [ ]:
from agentic.tools.memory import get_history, store_interaction, format_history_for_context

# Create a ticket for persistent memory
ticket_id = f"ticket-{uuid.uuid4().hex[:8]}"
print(f"Ticket ID: {ticket_id}")
print("=" * 60)

# Session 1
print("\n[SESSION 1]")
result1 = ask(
    "I need to cancel my reservation for the art gallery",
    thread_id=f"sess1-{ticket_id}",
    ticket_id=ticket_id
)
print(f"User: I need to cancel my reservation for the art gallery")
print(f"Agent: {result1['response'][:200]}...")

In [ ]:
# Check what's stored in long-term memory
print("\n📦 Long-Term Memory (TicketMessage table):")
print("-" * 40)

history = get_history(ticket_id, limit=10)
print(f"Messages stored: {len(history)}")
for msg in history:
    print(f"  [{msg['role'].upper()}]: {msg['content'][:60]}...")

In [ ]:
# Session 2 - NEW session, SAME ticket (memory should persist)
print("\n[SESSION 2 - New session, same ticket]")
result2 = ask(
    "Actually, can you remind me what we discussed?",
    thread_id=f"sess2-{ticket_id}",  # Different session
    ticket_id=ticket_id  # Same ticket = access to history
)
print(f"User: Actually, can you remind me what we discussed?")
print(f"Agent: {result2['response'][:300]}...")

In [ ]:
# Final memory state
print("\n📦 Final Long-Term Memory State:")
print("-" * 40)

history = get_history(ticket_id, limit=10)
print(f"Total messages: {len(history)}")
print("\nFormatted context:")
print(format_history_for_context(history))

---
## 6. End-to-End Workflow with Full Logging

Complete run showing: classification → routing → KB retrieval → tool usage → resolution/escalation

In [ ]:
print("=" * 60)
print("END-TO-END RUN: Resolution Path")
print("=" * 60)
print("\n📝 Query: 'How do I book an experience with my premium subscription?'\n")
print("Watch the structured JSON logs below:\n")

result = ask(
    "How do I book an experience with my premium subscription?",
    thread_id="e2e-resolve"
)

print("\n" + "=" * 60)
print("FINAL RESULT")
print("=" * 60)
print(f"Category: {result['category']}")
print(f"Confidence: {result['confidence']:.0%}")
print(f"Escalated: {result['escalated']}")
print(f"Tools: {result['tools_used']}")
print(f"\nResponse:\n{result['response']}")

In [ ]:
print("\n" + "=" * 60)
print("END-TO-END RUN: Escalation Path (Blocked Account)")
print("=" * 60)
print("\n📝 Query: 'My account has been blocked!'\n")
print("Watch the structured JSON logs below:\n")

result = ask(
    "My account has been blocked!",
    thread_id="e2e-escalate"
)

print("\n" + "=" * 60)
print("FINAL RESULT")
print("=" * 60)
print(f"Category: {result['category']}")
print(f"Escalated: {result['escalated']}")
print(f"\nResponse:\n{result['response']}")

---
## 7. RAG Confidence Scoring Demo

Direct test of similarity scores from Chroma.

In [ ]:
from agentic.tools.rag_tools import search_knowledge_with_scores

test_queries = [
    "How do I reset my password?",
    "What's included in the premium tier?",
    "I want a refund",
    "Enterprise API integration for webhooks",  # Should be low
    "Quantum computing partnership opportunities",  # Should be very low
]

print("RAG Confidence Scores (Real Chroma Similarity)")
print("=" * 60)
print(f"\n{'Query':<50} {'Confidence':<12} {'Action'}")
print("-" * 70)

for query in test_queries:
    _, confidence = search_knowledge_with_scores(query)
    action = "✅ Resolve" if confidence >= CONFIDENCE_THRESHOLD else "⚠️ Escalate"
    print(f"{query[:48]:<50} {confidence:.0%}{'':>8} {action}")

print(f"\nThreshold: {CONFIDENCE_THRESHOLD:.0%}")

---
## Summary

### Demonstrated:
1. ✅ **Tool Usage**: `get_user_info`, `get_user_reservations` invoked during workflow with logging
2. ✅ **Edge-Case**: Low confidence KB match → automatic escalation
3. ✅ **Session Memory**: Inspection of stored state by `thread_id`
4. ✅ **Long-Term Memory**: Cross-session persistence via `ticket_id`
5. ✅ **Structured Logging**: JSON format with node/event/data fields
6. ✅ **Real Confidence Scores**: From Chroma similarity (not string heuristics)

### Architecture
```
User Message
     ↓
CLASSIFIER → ACCOUNT_LOOKUP → RAG → [RESOLVER or ESCALATION]
     ↓              ↓           ↓
  category     db_tools     confidence
                            scoring
```